# 2.1 웹 크롤링으로 기초 데이터 수집하기
이번 절에서는 '나무위키 최근 변경 페이지'의 텍스트 데이터를 웹 크롤링으로 수집한 다음, 데이터 내에서 등장한 키워드의 출현 빈도를 분석해보겠습니다. 이를 통해 우리는 나무위키 페이지에서 현재 가장'핫한'키워드가 무엇인지 분석할 수 있습니다.<br>
웹 크롤링 혹은 웹 스크래핑은 인터넷에 있는 웹 페이지를 방문해서 페이지의 자료를 자동으로 수집하는 작업을 의미합니다. 이책에서는 파이썬으로 웹 크롤링을 하겠습니다.

## 대상 페이지 구조 살펴보기
크롤링을 위한 첫 번째 단계는 인터넷 익스플로러, 크롬 등의 웹 브라우저를 실행하여 크롤링의 대상이 될 페이지 구조를 살펴보는 것입니다. 먼저 웹 브라우저의'개발자 도구'를 실행합니다. 크롬의 경우, 아래 그림처럼 브라우저 우측 상단에 있는 점 3개의 모양의 [Chrome 맞춤설정 및 제어] 아이콘을 클릭합니다. 그리고 [도구 더보기]-[개발자 도구]를 순서대로 클릭하여 개발자 도구를 실행합니다. 혹은 단축키로 실행합니다(윈도우 : Ctrl+Shift+I, 맥OS : Command + Alt + I)|

## 웹 크롤링 라이브러리 사용하기
파이썬에서는 BeautifulSoup과  requests라는 라이브러리로 웹 크롤러를 만들 수 있습니다. requests는 특정 URL로부터 HTML 문서를 가져오는 작업을 수행하고, BeautifulSoup 모듈은 HTML 문서에서 데이터를 추출하는 작업을 수행합니다. 이 모듈들을 사용하기에 앞서, 터미널(cmd) 혹은 아나콘다 프롬프트를 실행하여 아래의 세 가지 파이썬 모듈을 설치해 주어야 합니다.
- [Anaconda Powershell Prompt]
- (pybook) c:\Users\yoonk> pip install lxml beautifulsoup4 requests

다음 코드에서 requests.get() 함수로 URL의 HTML 문서를 가져온 뒤, 이를 BeautifulSoup() 클래스의 soup 객체로 변환합니다. 그리고 find(), find_all() 함수를 사용하여 특정 HTML 택그 혹은 특정 HTML 클래스를 가진 데이터를 가져옵니다.

In [27]:
import requests
from bs4 import BeautifulSoup
import re

#크롤링할 사이트 주소를 정의합니다.
source_url= "https://namu.wiki/RecentChanges"

# 사이트의 HTML 구조에 기반하여 크롤링을 수행합니다.
req=requests.get(source_url)
html=req.content
soup=BeautifulSoup(html,'lxml')
contents_table=soup.find(name="table")
table_body=contents_table.find(name="tbody")
table_rows=table_body.find_all(name='tr')

# a 태그의 href 속성을 리스트로 추출하여 크롤링할 페이지 리스트를 생성합니다.
page_url_base="https://namu.wiki"
page_urls=[]
for index in range(0,len(table_rows)):
    first_td=table_rows[index].find_all('td')[0]
    td_url=first_td.find_all('a')
    if len(td_url) > 0:
        page_url=page_url_base+td_url[0].get('href')
        pagd_urls.append(page_url)
        
# 중복 url을 제거합니다.
page_urls=list(set(page_urls))
for page in page_urls[:5]:
    print(page)

AttributeError: 'NoneType' object has no attribute 'find'

위의 코드는 개발자 도구로 살펴본 HTML 구조에 기반하여 table > tbody > tr > td > a 태그 순의 HTML 꼐층 구조를 좁혀나가는 과정입니다. 이 과정을 통해 목표 태그에 도달했을 때, get(href) 함수로 태그의 속성 정보를 추출합니다. get() 함수는 해당 태그가 가지고 있는 특정한 속성을 추출합니다.

## 텍스트 정보 수집하기
아래는 Step1~Step2에서 추출한 웹 페이지들의 URL을 방문하여 HTML 구조를 개발자 도구로 살펴본 것입니다. 이페이지의 구조를 파악하여 텍스트 정보를 추출해보겠습니다.

이전과 마찬가지로 개발자 도구의 마우스 포인터 모양 아이콘을 클립합니다. 그리고 문서의 '제목'부분, 문서의'카테고리'부분, 그리고'본문' 부분을 클릭하여 HTML의 구조를 파악합니다. 문서의 전체 내용은 'article'이라는 태그 안에 구성되어 있습니다. 그리고 제목은 'h1'태그, 카테고리 부분은 'ul'태그 영역 안에 존재하며, 본문의 내용은 'wiki-paragraph'라는 클래스로 구성된 'div'태그 안에 위치하고 있습니다.<br>
다음 코드는 최근 변경된 문성 중 한 페이지의 텍스트 정보를 크롤링한 것입니다. 이전 단계와 다른 점은 get() 함수 대신 text()함수를 사용하여 태그의 텍스트 정보만 추출했다는 점입니다.

#### URL 페이지 정보를 기반으로 크롤링하기

In [33]:
# 하나의 최근 변경된 문서를 크롤링합니다.
req=requests.get(page_urls[0])
html=req.content
soup=BeautifulSoup(html, 'lxml')
contents_table=soup.find(name="article")
title=contents_table.find_all('h1')[0]
category=contents_table.find_all('ul')[0]
content_paragraphs=contents_table.find_all(name="div", attrs={"class":"wiki-para-graph"})
content_corpus_list=[]

# 크롤링한 문서 정보 출력
for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)
content_corpus="".join(content_corpus_list)

print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

NameError: name 'page_urls' is not defined

# 2.2 나무위키 최근 변경 페이지 키워드 분석하기
이제 분석에 사용할 데이터가 준비되었으니, 본격적으로 텍스트 마이닝을 알아봅시다.
## 분석 미리보기
## 나무위키로 알아보는 실시간 지식정보 트렌드
아래 그림은 이번 절의 최종 분석 결과인 '키워드 출현 빈도수 분석'을 시각화 한것입니다. 이를 통해 현재 나무위키 페이지에서 어떤 키워드들이 활발하게 수정되고 있는지를 한눈에 파악할 수 있습니다.

## Step1 크롤링: 웹데이터 가져오기
이전 단계와 동일한 방법으로 웨 데이터를 크롤링합니다. 단, 이번에는 모든 URL의 데이터를 가져와 봅시다.<br>
다음 코드를 실행하여 나무위키에서 최근 변경이 일어난 페이지들의 URL을 page_urls라는 변수에 저장합니다.

#### BeautifulSoup을 이용해 웹 크롤링하기

In [37]:
# -*- coding: uft-8

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re

# 크롤링할 싸이트 주소를 정의합니다.
soure_url= "https://namu.wiki/RecentChanges"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
req=requests.get(source_url)
html=req.content
soup=BeautifulSoup(html,'lxml')
contents_table=soup.find(name="table")
table_body=contents_table.find(name="tbody")
table_rows=table_body.find_all(name='tr')

# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_url_base= "https://namu.wiki"
page_urls=[]
for index in range(0, len(table_rows)):
    first_td=table_rows[index].find_all('td')[0]
    td_url=first_td.find_all('a')
    if len(td_url) > 0:
        page_url=paga_url_base+td_url[0].get('href')
        if 'png' not in page_url:
            page_urls.append(page_url)
page_urls=list(set(page_urls))

AttributeError: 'NoneType' object has no attribute 'find'

이제 이 주소들에 다시 한번 접근하여 문서의 본문과 제목, 그리고 카테고리에 등장하는 텍스트 데이터를 가져와봅시다.

#### 나무위키의 최근 변경 데이터 크롤링하기

In [41]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns=['title','category','content_text']
df=pd.DataFrame(columns=columns)

# 각 페이지별 '제목','카테고리','본문' 정보를 데이터 프레임으로 만듭니다.
for page_url in page_urls:
    
    # 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
    req=requests.get(page_url)
    html=req.content
    soup=BeauifulSoup(html,'lxml')
    contents_table=soup.find_all('h1')[0]
    category=contents_table.find_all('ul')[0]
    content_paragraphs=contents_table.find_all(name='div',attrs={"class":"wiki-paragraph"})
    content_corpus_list=[]
    
    # 페이지 내 제목 정보에서 개행 문자를 제거한 뒤 추출합니다.
    # 만약 없는 경우, 빈 문자열로 대체합니다.
    if title is not None:
        row_title=title.text.replace("\n","")
    else:
        row_title=""
        
    # 페이지 내 본문 정보에서 개행 문자를 제거한 뒤 추출합니다.
    # 만약 없는 경우, 빈 문자열로 대체합니다.
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n",""))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
        
    # 페이지 내 카테고리정보에서 "분류"라는 단어와 개행 문자를 제거한 뒤 추출합니다.
    # 만약 없는 경우, 빈 문자열로 대체합니다.
    if category is not None:
        row_category = category.text.replace("\n","")
    else:
        row_category=""
        
    # 모든 정보를 하나의 데이터 프레임에 저장합니다.
    row=[row_title, row_category, "".joun(content_corpus_list)]
    series=pd.Series(row, index=df.columns)
    df=df.append(series,ignore_index=True)
    
# 데이터 프레임을 출력합니다.
df.head(5)

NameError: name 'page_urls' is not defined

위의 실행 결과는 모든 URL의 텍스트 데이터를 가져온 뒤, 이를 데이터 프레임의 형태로 변환한 것입니다. 데이터에 등장하는 불필요한 문자인'\n','분류'는 크롤링 과정에서 replace()함수로 제거해주었습니다.

## Step2 추출: 키워드 정보 추출하기
다음은 수집한 텍스트 데이터에서 키워드 정보를 추출하는 단계입니다. 이를 텍스트 전처리 작업이 필요합니다. 텍스트 전처리는 특수문자나 외국어를 제거하는 등의 과정을 포함합니다. 그런데 이는 언어와 상황마다 조금씩 다를 수 있습니다.<br>
예를 들어 스팸메일을 분류하는 텍스트 마이닝의 경우, 특수문자나 외국어가 분석의 중요한 힌트가 될 수 있기 때문에 이를 제거하지 않는 편입니다. 반면, 키워드 분석처럼 '단어'를 추출하는 것이 목적이라면 특정 언어의 글자만을 추출하기도 합니다.<br>
파이썬에서 're'라는 모듈을 통해 정규표현식을 사용할 수 있습니다. 정규표현식이란 특정한 규칙을 가진 문자열의 집합을 표현하는 형식입니다. 만약 다음 코드와 같이 re.compile('^ㄱ-ㅣ가-힣]+')이라는 코드로 한글에 대한 정규표현식을 정의하면 대상이 되는 텍스트 데이터에서 한글만 추출할 수 있게 됩니다.

#### 텍스트 데이터 전처리하기

In [42]:
# 텍스트 정제 함수: 한글 이외의 문자는 전부 제거합니다.
def text_cleaning(text):
    hangul=re.compile('^ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result=hangul.sub('',text)
    return

print(text_cleaning(df['content_text'][0]))

IndexError: index 0 is out of bounds for axis 0 with size 0

모든 데이터에 전처리를 적용하기 위해서는 apply() 함수를 사용합니다. 다음 코드는 title,category, content_text 3개의 피처에 apply() 함수를 적용한 것입니다. 이를 head() 함수로 출력하면 한글을 제외한 문자들이 제거된 것을 확인할 수 있습니다.

#### 모든 데이터에 전처리 적용하기

In [43]:
# 각 피처마다 데이터 전처리를 적용합니다.
df['title']= df['title'].apply(lambda x : text_cleaning(x))
df['category']=df['category'].apply(lambda x: text_cleaning(x))
df['content_text']=df['content_text'].apply(lambda x: text_cleaning(x))
df.head(5)

,title,category,content_text


다음 과정은 키워드를 추출한 뒤, 빈도 분석을 수행하는 과정입니다. 여기서 키워드를 추출한다는 것은 무엇을 의미할까요? 키워드 추출이란 좁은 의미에서는 명사, 혹은 형태소 단위의 문자열을 추출하는 것입니다. 이를 수행하기 위해 말뭉치라는 것을 만들어야 합니다.<br>
말뭉치는 말 그대로 텍스트 데이터의 뭉텅이를 의미합니다. 이번 예제에서는 제목 단위, 카테고리 단뒤, 본문 단위의 키워드를 분석하기 위해 제목 말뭉치, 카테고리 말뭉치, 본문 말뭉치 총 3개의 말뭉치를 생성합니다<br>
다음 코드에서는 텍스트 피처를 tolist()로 추출한 뒤, join() 함수로 말뭉치를 생성해주었습니다.
실행결과는 제목 말뭉치의 출력 결과입니다.

#### 말뭉치 만들기

In [45]:
# 각 피처마다 말뭉치를 생성합니다.
title_corpus="".join(df['title'].tolist())
category_corpus="".join(df['category'].tolist())
content_corpus="".join(df['content_text'].tolist())
print(title_corpus)

이제 각 말뭉치 안에서 등장하는 형태소를 추출하겠습니다. 파이썬의 한국어 형태소 추출 라이브러리 중 가장 쉽게 사용할 수 있는 konlpy를 사용합니다.